In [1]:
import pandas as pd
import numpy as np
import torch

from model_3d.Schnet_NMR import SchNet

from load_Glycosciencedb_3d import create_graph_experiment

from load_GODDESS_3d import GODDESSDataset, split_test_val
from tqdm import tqdm
from run_fine_tune import run_tune

from run_no_val import run
np.random.seed(9721)

In [2]:
num_test = 270

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Create = create_graph_experiment(data_dir='glycosciencedb/data_reformulate/',
                                 atom_embed_dir='merged_embed/atom_name_embed.csv',
                                 mono_embed_dir='merged_embed/monosaccharide_embed.csv',
                                 ab_embed_dir='merged_embed/ab_embed.csv',
                                 dl_embed_dir='merged_embed/dl_embed.csv',
                                 pf_embed_dir='merged_embed/pf_embed.csv',
                                 num_test=num_test, seed=97211)

train_data_exp, test_data_exp = Create.create_all_graph_list()

loss_func = torch.nn.L1Loss()
run3d_tune = run_tune()

node_embedding_size = train_data_exp[0].z.shape[1]

--------------------------loading NMR Graph List-------------------------------


100%|█████████████████████████████████████████| 299/299 [00:07<00:00, 39.69it/s]


In [3]:
b = 2
h = 256
c = 5.0
n = 2
l = 0.001

In [4]:
run3d_final= run()

model = SchNet(hidden_channels = h, cutoff = 5, num_layers=n, in_embed_size = node_embedding_size)


In [5]:
model.load_state_dict(torch.load('transfer_learning_results/3D_godess_schnet/Model_Godess_carbon.pt'))

<All keys matched successfully>

In [6]:
for name, para in model.named_parameters():
#     print("-"*20)
    print(name)
    if name not in ['update_u.lin1.weight', 'update_u.lin1.bias', 'update_u.lin2.weight', 'update_u.lin2.bias',
                    'update_es.1.lin.weight', 'update_es.1.mlp.0.weight','update_es.1.mlp.0.bias', 
                    'update_es.1.mlp.2.weight','update_es.1.mlp.2.bias', 'update_vs.1.lin1.weight',
                    'update_vs.1.lin1.bias','update_vs.1.lin2.weight','update_vs.1.lin2.bias']:
        para.requires_grad = False

init_v.weight
init_v.bias
update_vs.0.lin1.weight
update_vs.0.lin1.bias
update_vs.0.lin2.weight
update_vs.0.lin2.bias
update_vs.1.lin1.weight
update_vs.1.lin1.bias
update_vs.1.lin2.weight
update_vs.1.lin2.bias
update_es.0.lin.weight
update_es.0.mlp.0.weight
update_es.0.mlp.0.bias
update_es.0.mlp.2.weight
update_es.0.mlp.2.bias
update_es.1.lin.weight
update_es.1.mlp.0.weight
update_es.1.mlp.0.bias
update_es.1.mlp.2.weight
update_es.1.mlp.2.bias
update_u.lin1.weight
update_u.lin1.bias
update_u.lin2.weight
update_u.lin2.bias


In [7]:
import time
start_time = time.time()

train_loss_list, test_rmse_list = run3d_final.run(device=device, train_dataset = train_data_exp,
                                                  test_dataset = test_data_exp,
                                                  model = model, loss_func=loss_func,
                                                  epochs=50, batch_size=b, vt_batch_size= 2, lr=l, lr_decay_factor=0.5, lr_decay_step_size=15)
print("--- %s seconds ---" % (time.time() - start_time))

#Params: 489985

=====Epoch 1

Training...


100%|███████████████████████████████████████████| 15/15 [00:00<00:00, 19.48it/s]



Testing...



100%|████████████████████████████████████████| 135/135 [00:00<00:00, 214.73it/s]


{'Train': 5.230003054936727, 'Test': 9.961835}

=====Epoch 2

Training...


...


{'Train': 0.6464278707901637, 'Test': 4.7667904}

=====Epoch 50

Training...



100%|██████████████████████████████████████████| 15/15 [00:00<00:00, 180.97it/s]



Testing...



100%|████████████████████████████████████████| 135/135 [00:00<00:00, 234.30it/s]


{'Train': 0.6237762441237767, 'Test': 4.7791977}
Test MAE when got best validation result: 4.763859272003174
--- 35.37581777572632 seconds ---


In [9]:
# schnet

sch_result = np.array([4.830, 4.974, 4.753, 4.749, 4.763])
np.mean(sch_result), np.std(sch_result)

(4.8138000000000005, 0.08532619761831663)